In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
%matplotlib inline
#%matplotlib notebook
sns.set(rc={'figure.figsize':(11.7,8.27)})
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ks_2samp
import scipy.stats as stats
from scipy.stats import ttest_rel

In [2]:
data = pd.read_pickle('df_tesco_marker_V3.pkl')

In [4]:
data2 = data.query('volumeADO >=100')
data2['logVolAdo'] =   np.log(data2['volumeADO'])
data2['logP_by_P0_ADO'] = np.log(data2['ADO_ Approved Price']/data2['ADO_ Marker Price'])
data2['logVolUL95'] =    np.log(data2['volumeUL95'])
data2['logP_by_P0_UL95'] = np.log(data2['UL 95_ Approved Price']/data2['UL 95_ Marker Price'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [110]:
#elasticity calculation :: forcing the intercept to zero 
def regRes_gen_const(site,x,y,prefix):
    cols = ['const','slope','rvalu','pvalue','count','mape_err']
    cols_ref = list(map(lambda x: x+prefix,cols))
    try:
        # filter all data of the site between 5th & 95th precentile 
        #q_low,q_up = site[y].quantile([0.02,0.98]).values.round().tolist()
        #xf = site.loc[(site[y] >= q_low)& (site[y] <= q_up)]

        #q_low,q_up = site[x].quantile([0.02,0.98]).values.round().tolist()
        #xf = site.loc[(site[x] >= q_low)& (site[x] <= q_up)]

        x_train,y_train = site[x], site[y]
        x_train = sm.add_constant(x_train)

        model = sm.OLS(y_train,x_train).fit()
        const,slope,rvalu,pvalue = model.params[0],model.params[1],round(model.rsquared,2),model.pvalues[1]
        
        mape_train_err = np.mean(abs(site[y] -(const + site[x]*slope))*100/site[y])
        #slope,intercept,rvalu,pvalue,stderr= stats.linregress(xf[x],xf[y])

        return(pd.Series([const,slope,rvalu,pvalue,len(x_train),mape_train_err],index=cols_ref))
    except Exception as e:
        print(e)
        return(pd.Series([0,0,0,0,len(x_train)],index=cols_ref))

In [111]:
# fuel sales volume wrt ADO price position 
x_var = 'logP_by_P0_ADO'
ref = '_ADO'
data3 = data2.query('logP_by_P0_ADO != 0')[['Site Name','PBL','logVolAdo','logP_by_P0_ADO']].reset_index()
regFuelVolEl_ADO = data3.groupby(['Site Name','PBL']).apply(lambda site: \
                                                regRes_gen_const(site,x_var,'logVolAdo',ref)).reset_index()

In [113]:
# fuel sales volumne wrt UL
x_var = 'logP_by_P0_UL95'
ref = '_UL'
data3 = data2.query('logP_by_P0_UL95 != 0')[['Site Name','PBL','logVolUL95','logP_by_P0_UL95']].reset_index()
regFuelVolEl_UL = data3.groupby(['Site Name','PBL']).apply(lambda site: \
                                                regRes_gen_const(site,x_var,'logVolUL95',ref)).reset_index()

In [14]:
post_march_2020 = pd.read_pickle(r'post_march_2020.pkl')

rel_df = post_march_2020.loc[:,['SiteName_x','PBL','SalesTransactionVolumeADO', 'SalesTransactionVolumeUL95'\
                              ,'ApprovedPriceUL95', 'MarkerPriceADO', 'ApprovedPriceADO','MarkerPriceUL95']]

rel_df = rel_df.query('SalesTransactionVolumeADO > 100 & SalesTransactionVolumeUL95 > 100')

rel_df['log_volADO'] =   np.log(rel_df['SalesTransactionVolumeADO'])
rel_df['log_volUL95'] =   np.log(rel_df['SalesTransactionVolumeUL95'])

rel_df['logP_by_P0_ADO'] = np.log(rel_df['ApprovedPriceADO']/rel_df['MarkerPriceADO'])
rel_df['logP_by_P0_UL95'] = np.log(rel_df['ApprovedPriceUL95']/rel_df['MarkerPriceUL95'])

In [114]:
rel_df_ADO = rel_df.merge(regFuelVolEl_ADO[[ 'PBL', 'const_ADO', 'slope_ADO','mape_err_ADO']],how='left',on=['PBL'])
rel_df_ADO_UL =  rel_df_ADO.merge(regFuelVolEl_UL[[ 'PBL', 'const_UL', 'slope_UL','mape_err_UL']],how='left',on=['PBL'])

rel_df_ADO_UL['pred_log_volADO'] = rel_df_ADO_UL.const_ADO + \
                                                rel_df_ADO_UL.slope_ADO*rel_df_ADO_UL.logP_by_P0_ADO
    
rel_df_ADO_UL['pred_log_volUL95'] = rel_df_ADO_UL.const_UL + \
                                                rel_df_ADO_UL.slope_UL*rel_df_ADO_UL.logP_by_P0_UL95


rel_df_ADO_UL['mape_fuelSales_ADO'] = (abs(rel_df_ADO_UL.log_volADO - rel_df_ADO_UL.pred_log_volADO)\
                                       /rel_df_ADO_UL.log_volADO)*100

rel_df_ADO_UL['mape_fuelSales_UL'] = (abs(rel_df_ADO_UL.log_volUL95 - rel_df_ADO_UL.pred_log_volUL95)\
                                      /rel_df_ADO_UL.log_volUL95)*100

In [119]:
rel_df_ADO_UL.shape

(21907, 22)

In [116]:
rel_df_ADO_UL_mape = rel_df_ADO_UL.groupby('SiteName_x')[['mape_err_ADO','mape_err_UL',\
                                                          'mape_fuelSales_ADO','mape_fuelSales_UL']].mean()

In [117]:
rel_df_ADO_UL_mape.to_csv(r'rel_df_ADO_UL_mape.csv')